# Try out Pandas for gauge data

In [1]:
%pylab inline

In [2]:
import pandas as pd

In [3]:
column_names = ['gaugeno', 'level', 'time', 'h', 'hu', 'hv', 'eta'] # for geoclaw

In [4]:
gdata = pd.read_csv('data/fort.gauge', delim_whitespace=True,
                    names=column_names, header=None)
gdata.shape

In [5]:
gdata[:6]

In [6]:
gdata.tail(3)

### Extract DART data:

In [7]:
dart = gdata[gdata['gaugeno']==32412]

In [8]:
dart.tail(3)

In [9]:
eta = dart['eta']  # or equivalently, dart.eta
print "dart['eta'] has type %s" % type(eta)
etamin = eta.min()
etamax = eta.max()
print "At the DART gauge, eta varies between %6.3f and %6.3f meters" % (etamin,etamax)

In [10]:
# the indices of eta are still relative to the original gdata:
eta.index

In [11]:
print "The shape of eta is %s and the last three elements are:" % eta.shape
eta[-3:]

In [12]:
eta_np = array(eta)
print "After converting to a numpy array, the shape of eta_np is %s" % eta_np.shape
print "The last three elements are:", eta_np[-3:]

In [13]:
figure(figsize=(5,3))
plot(dart['time']/3600., dart['eta'],'b')
title('Surface eta at DART gauge 32412')
xlabel('Hours since event')

### Plot eta at all gauges:

In [49]:
gaugenos = list(set(gdata.gaugeno))
gaugenos.sort()
print "Gauge numbers in gdata: %s" % gaugenos
figure(figsize=(6,4))
c = ['b','g','r']  # colors for different gauges
for k,gaugeno in enumerate(gaugenos):
    data = gdata[gdata.gaugeno==gaugeno]
    plot(data['time']/3600., data['eta'],color=c[k],label='Gauge %s' % gaugeno)
    legend()
    title('Surface eta')
    xlabel('Hours since event')

### Examine AMR levels in DART gauge data:

In [15]:
levels = list(set(dart['level']))
levels.sort()
print "DART gauge has data at levels: %s" % levels
for levelno in levels:
    data = dart[dart['level'] == levelno]
    print "Level %s first appears at t = %5.2f hours" % (levelno, data['time'].min()/3600.)

In [16]:
figure(figsize=(6,4))
c = ['b','g','r']  # colors for different levels
for levelno in [1,2,3]:
    data = dart[dart['level'] == levelno]
    plot(data['time']/3600., data['eta'],color=c[levelno-1],\
         linestyle='None',marker='o',label='Level %s' % levelno)
legend()

## Working with datetime

In [17]:
import datetime

In [18]:
year = 2010
month = 2
day = 27
hour = 6
minute = 34
event_time = datetime.datetime(year, month, day, hour, minute)
print event_time

In [19]:
time_since_event = [event_time + datetime.timedelta(seconds=s) for s in gdata['time']]

In [20]:
gdata['real_time'] = time_since_event  # adds a new column
gdata[:6]

In [21]:
dart = gdata[gdata['gaugeno']==32412]

In [22]:
fig = figure(figsize=(9,4))
ax = axes()
ax.plot_date(dart['real_time'], dart['eta'],'-')
ax.xaxis.set_major_locator(HourLocator())
ax.xaxis.set_minor_locator(MinuteLocator(arange(0,61,15)))
ax.xaxis.set_major_formatter(DateFormatter('%H:%M'))
fig.autofmt_xdate()
title('Event time: %s' % event_time.strftime('%Y-%m-%d %H:%M:%S'))
show()

## Test adding column names after reading:

In [24]:
gdata = pd.read_csv('data/fort.gauge',delim_whitespace=True,header=None,names=None)

In [25]:
gdata[:3]

In [26]:
gdata.columns = column_names

In [27]:
gdata[:3]

## Merge two overlapping time series

This will be useful when more than one `fort.gauge` file exists from an original run and a restart and the series must be merged together.  

To try this out, first create two overlapping time series by subsampling the `dart` data:

In [44]:
dart1 = dart.copy()[:400]
print "dart1 has %s elements from t = %8.1f to t = %8.1f" \
     % (len(dart1.eta), dart1.time.min(), dart1.time.max())

In [45]:
dart2 = dart.copy()[200:]
print "dart2 has %s elements from t = %8.1f to t = %8.1f" \
     % (len(dart2.eta), dart2.time.min(), dart2.time.max())

### Plot them together to see the overlap:

In [46]:
plot(dart1['time']/3600, dart1['eta'],'b',linewidth=2)
plot(dart2['time']/3600, dart2['eta'],'r--',linewidth=2)

In [47]:
# recombine:
dart_reconstructed = dart1.combine_first(dart2)
dart_reconstructed.shape

In [48]:
plot(dart_reconstructed['time']/3600, dart_reconstructed['eta'])